# Figure 1: Complexity Experiment (Covering vs. Optimization)

The code below reproduces the complexity comparison between a covering method and an optimization-based search method for invariance. It uses the background-modeling registration optimization formulation (eqn (4) in arXiv v1 paper), with various motion models. Covering is simulated by random sampling.

The implementation is in numpy.

In [ ]:
# Handle imports
import sys
sys.path.append("../src")

import time
import os
import numpy as np
from registration_exp import (generate_data, dilate_support, get_default_pgd_dict, registration_l2_exp)
import pickle

In [ ]:
# Helper function: set up a single run of the complexity experiment (one motion model)
def test_complexity_textured_run_data(mode, sigma_init=5, multiscale=7, ntry_cover=10000):
# mode = 'affine', 'similarity', 'euclidean' or 'translation'
#
# Here we set the maximum number of tries for covering as 10,000.
# More tries are required for transform modes of higher dimension.
# Can manually run for more tries.



    vecnorm_2 = lambda A: np.linalg.norm( A.ravel(), 2 )


    if not os.path.isdir('exp_{:s}/'.format(mode)):
        os.mkdir('exp_{:s}/'.format(mode))


    for idx in range(1,11):
        print('\n===== {} =====\n'.format(idx))
        
        X0, X, Om, c, tf_params, extents = generate_data(mode)
        with open('exp_{:s}/{:02d}_data.pkl'.format(mode,idx), "wb") as f:
            pickle.dump([X0 ,X, Om ,c, tf_params], f)
        
        
        
        
        # Optimization
        t = time.time()

        if mode == 'affine':
            A = np.eye(2)
            b = np.zeros((2,))
            optim_vars = [A, b]
        elif mode == 'similarity':
            dil = 1
            phi = 0
            b = np.zeros((2,))
            optim_vars = [dil, phi, b]
        elif mode == 'euclidean':
            phi = 0
            b = np.zeros((2,))
            optim_vars = [phi, b]
        elif mode == 'translation':
            b = np.zeros((2,))
            optim_vars = [b]

        sigma = sigma_init

        for repeat in range(multiscale):
            Om_tilde = dilate_support(Om,sigma)
            param_dict = get_default_pgd_dict(
                step=0.005*sigma, max_iter=50, epoch_len=10, parametric=True, center=c, tol=1e-5,
                sigma=sigma, quiet=False, use_nesterov=False )
            tau_u, tau_v, optim_vars, error, Rvals = registration_l2_exp( X, X0, Om_tilde, Om, c,
                                                                       mode, optim_vars, param_dict=param_dict,
                                                                       visualize=False)

        elapsed = time.time() - t
        print('\nTime: {:.1f}\n\n'.format(elapsed/60))

        with open('exp_{:s}/{:02d}_optim.pkl'.format(mode,idx), "wb") as f:
            pickle.dump([Rvals, error, elapsed, optim_vars], f)



        
        # Covering
        t = time.time()

        maxTry = ntry_cover
        numTry = 0
        bestR = 0
        Rvals_cover = np.zeros(maxTry)

        cur_X = X0
        cur_X_wd = cur_X * Om
        for ic in range(3):
            cur_X_wd[:,:,ic] -= np.mean(cur_X_wd[:,:,ic][cur_X_wd[:,:,ic] > 0])

        while bestR < .95 and numTry < maxTry: 

            if (numTry+1) % 1000 == 0:
                print([numTry//1000, bestR])

            [s_dist,phi_dist,theta_dist,b_dist] = extents
            [A,b,dil,phi] = sample_random_transform(mode, s_dist,phi_dist,theta_dist,b_dist)

            A_inv = np.linalg.inv(A)
            b_inv = - A_inv @ b 
            X_try, Om_try = apply_affine_transform(X, Om, c, A, b)

            cur_Y_wd = X_try * Om
            for ic in range(3):
                cur_Y_wd[:,:,ic] -= np.mean(cur_Y_wd[:,:,ic][cur_Y_wd[:,:,ic] > 0])

            curR = np.sum(Om * cur_X_wd * cur_Y_wd) / ( vecnorm_2(Om * cur_X_wd) * vecnorm_2(Om * cur_Y_wd) )
            Rvals_cover[numTry] = curR

            bestR = max(bestR,curR)
            numTry += 1

        Rvals_cover = Rvals_cover[:numTry]

        print(f'\nNumTried {numTry}   BestR {bestR}')    

        elapsed = time.time() - t
        print('\nTime: {:.1f}\n\n'.format(elapsed/60))

        with open('exp_{:s}/{:02d}_cover.pkl'.format(mode,idx), "wb") as f:
            pickle.dump([Rvals_cover, elapsed], f)



In [ ]:
test_complexity_textured_run_data('translation')
test_complexity_textured_run_data('euclidean')
test_complexity_textured_run_data('similarity')
test_complexity_textured_run_data('affine', sigma_init=10)

target_corr = 0.9

iter_recs = np.zeros((2,4))
# time_recs = np.zeros((2,4))
incomplete = np.zeros((2,4))

for idx in range(1,11):
    
    with open('exp_affine/{:02d}_optim.pkl'.format(idx), "rb") as f:
        [Rvals_optim, _, elapsed_optim, _] = pickle.load(f)
    with open('exp_affine/{:02d}_cover.pkl'.format(idx), "rb") as f:
        [Rvals_cover, elapsed_cover] = pickle.load(f)
    
    good_id = np.where(Rvals_optim > target_corr)[0]
    if len(good_id) > 0:
        iter_recs[0,0] += good_id[0] + 1
        # time_recs[0,0] += (good_id[0]+1) / len(Rvals_optim) * elapsed_optim
    else:
        incomplete[0,0] = 1
        
    good_id = np.where(Rvals_cover > target_corr)[0]
    if len(good_id) > 0:
        iter_recs[1,0] += good_id[0] + 1
        # time_recs[1,0] += (good_id[0]+1) / len(Rvals_cover) * elapsed_cover
    else:
        incomplete[1,0] = 1
        
        
    with open('exp_similarity/{:02d}_optim.pkl'.format(idx), "rb") as f:
        [Rvals_optim, _, elapsed_optim, _] = pickle.load(f)
    with open('exp_similarity/{:02d}_cover.pkl'.format(idx), "rb") as f:
        [Rvals_cover, elapsed_cover] = pickle.load(f)
    
    good_id = np.where(Rvals_optim > target_corr)[0]
    if len(good_id) > 0:
        iter_recs[0,1] += good_id[0] + 1
        # time_recs[0,1] += (good_id[0]+1) / len(Rvals_optim) * elapsed_optim
    else:
        incomplete[0,1] = 1
        
    good_id = np.where(Rvals_cover > target_corr)[0]
    if len(good_id) > 0:
        iter_recs[1,1] += good_id[0] + 1
        # time_recs[1,1] += (good_id[0]+1) / len(Rvals_cover) * elapsed_cover
    else:
        incomplete[1,1] = 1
        
        
    with open('exp_euclidean/{:02d}_optim.pkl'.format(idx), "rb") as f:
        [Rvals_optim, _, elapsed_optim, _] = pickle.load(f)
    with open('exp_euclidean/{:02d}_cover.pkl'.format(idx), "rb") as f:
        [Rvals_cover, elapsed_cover] = pickle.load(f)
    
    good_id = np.where(Rvals_optim > target_corr)[0]
    if len(good_id) > 0:
        iter_recs[0,2] += good_id[0] + 1
        # time_recs[0,2] += (good_id[0]+1) / len(Rvals_optim) * elapsed_optim
    else:
        incomplete[0,2] = 1
        
    good_id = np.where(Rvals_cover > target_corr)[0]
    if len(good_id) > 0:
        iter_recs[1,2] += good_id[0] + 1
        # time_recs[1,2] += (good_id[0]+1) / len(Rvals_cover) * elapsed_cover
    else:
        incomplete[1,2] = 1
        
        
    with open('exp_translation/{:02d}_optim.pkl'.format(idx), "rb") as f:
        [Rvals_optim, _, elapsed_optim, _] = pickle.load(f)
    with open('exp_translation/{:02d}_cover.pkl'.format(idx), "rb") as f:
        [Rvals_cover, elapsed_cover] = pickle.load(f)
    
    good_id = np.where(Rvals_optim > target_corr)[0]
    if len(good_id) > 0:
        iter_recs[0,3] += good_id[0] + 1
        # time_recs[0,3] += (good_id[0]+1) / len(Rvals_optim) * elapsed_optim
    else:
        incomplete[0,3] = 1
        
    good_id = np.where(Rvals_cover > target_corr)[0]
    if len(good_id) > 0:
        iter_recs[1,3] += good_id[0] + 1
        # time_recs[1,3] += (good_id[0]+1) / len(Rvals_cover) * elapsed_cover
    else:
        incomplete[1,3] = 1
        
        
        
iter_recs /= 10
time_recs /= 10
iter_recs = iter_recs[:, ::-1]
time_recs = time_recs[:, ::-1]

iter_recs[0,:] = iter_recs[0,:] * np.array([[4,4,5,8]])
        
print('Incomplete runs:')
print(incomplete)
print('Average complexity:')
print(iter_recs)
